In [ ]:
# the same housekeeping
import os

try:
    workdir
    raise Exception("This cell should be run only once (workdir = %s)" % workdir)
except NameError:
    pass
   
basedir=os.getcwd()
workdir=basedir + '/work'

In [ ]:
import mdtraj as md
import os

os.chdir(workdir)

tr = md.load('full-landmarks.pdb')
idx = tr[0].top.select("element != H")

heavy = tr.atom_slice(idx)
heavy.center_coordinates()
first = heavy.slice(0)


heavy.save_xtc('heavy-landmarks.xtc')
first.save_pdb('heavy1.pdb')




In [ ]:
import anncolvar

nlayers = 1
layers = [32, 0, 0]
actfun = ['sigmoid','linear','linear']
optim = 'adam'
loss = 'mean_squared_error'
epochs = 500
batch = 256

os.chdir(workdir)

rg = md.compute_rg(first)

# magic -- seems to be safe
box = rg[0] * 5

!gmx editconf -f heavy1.pdb -o reference.pdb -box {box} {box} {box} -c 

# XXX: too much stdout 
for col in [1,2]:
    anncolvar.anncollectivevariable('heavy-landmarks.xtc','reference.pdb','colvar.txt',col,
                                    box,box,box,.1,0,0,
                                    1,*layers,
                                    *actfun,
                                    optim,loss,epochs,batch,
                                    '','','plumed%d.dat' % col)
                                    
                                    

In [ ]:
# TODO: renumber atoms in plumed*.dat back and merge them